# Brain Age 20-90 Script

In [1]:
path=  'C:/Users/felip/OneDrive - Universidad Técnica Federico Santa María/Documentos/Brain/'
#import sys
#sys.path.append(path) 
path_=  'C:/Users/felip/OneDrive - Universidad Técnica Federico Santa María/Documentos/Brain/Brain_Age_Clock/'
import sys
sys.path.append(path_) 

# Regressor

In [ ]:
from sklearn.linear_model import Lasso
from base_regressor import BaseRegressor
from Plotter import Plotter
from skopt.space import Real, Categorical, Integer

class LassoRegressor(BaseRegressor):
    def __init__(self, save_path=None, scaler=None, params=None, params_space=None, fit_params_search=None, model_params_search=None, fit_params_train=None, model_params_train=None, name_model="Lasso"):
        super().__init__(save_path, scaler, params, params_space, fit_params_search, model_params_search, fit_params_train, model_params_train, name_model)

        self.model_ml = Lasso
        if params is None:
            self.params = {
                'alpha': 0.2,
                'max_iter': 10000,
                #'tol': 0.001
            }
        if params_space is None:
            self.params_space = {
                'alpha': Real(0.001, 10.0, prior='log-uniform'),
                'max_iter': Integer(100, 10000),
                'tol': Real(1e-5, 1e-2, prior='log-uniform')
            }


c:\python_env\brain_age_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Pipeline

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
def pipeline_regression_clocks(model_reg, df_concatenado_CN, Scaler_reg_train, Scaler_reg):
    # --------------------------- 1. Selección de Métricas ---------------------------
    X_CN = df_concatenado_CN.iloc[:, :-2]
    y_CN = df_concatenado_CN.iloc[:, -2]
    ID_CN = df_concatenado_CN.iloc[:, -1]

    Scaler_reg.fit(X_CN)
    X_CN_scaled = Scaler_reg.transform(X_CN)
    X_CN_scaled = pd.DataFrame(X_CN_scaled, columns=X_CN.columns)
    
    # --------------------------- 2. Búsqueda de Hiperparámetros ---------------------------
    opt_model, best_params = model_reg.search_best_model (X=X_CN_scaled, y=y_CN, n_iter_=50, scoring_metric='neg_mean_absolute_error')

    # --------------------------- 3. Entrenamiento con Controles ---------------------------
    results_labels_df_CN_train, results_labels_df_CN_test,  results_model, results_per_fold_CN_train, results_per_fold_CN_test, df_CN_avg_train,df_CN_avg_test= model.trainer(
        X=X_CN,
        y=y_CN,
        ID_label='ID-unique',
        ID=ID_CN,
        n_splits=10,
        n_iterations=20,
        params_=best_params
    )

    # --------------------------- 4. Shap ---------------------------
    shap_values_CN, shap_values_avg_CN, shap_summary_sorted_CN, shap_per_fold_CN = model_reg.calculate_multiple_shap(
    df_concatenado_CN, df_concatenado_CN,'ID-unique', results_per_fold_CN_train, results_per_fold_CN_test, results_model['model'],scaler=Scaler_reg_train)
    
    # --------------------------- 5. Concatenación de Resultados ---------------------------
   

    # --------------------------- Resultado Final ---------------------------
    return (
        columnas_por_region,
        (opt_models, best_params),
        (results_labels_CN_train, results_labels_CN_test, results_models, results_folds_CN_train, 
         results_folds_CN_test, results_avg_CN_train, results_avg_CN_test),
        (results_labels_test_Patient, results_folds_test_Patient, results_avg_test_Patient),
        (df_results_clocks_CN, df_results_clocks_AD, df_results_clocks_FTD)
    )
